In [50]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.cluster import contingency_matrix
os.chdir('C:\\Users\\prudi\\Desktop\\Data Sets\\Loan Prediciton_AnalyticsVidhya\\train_aox2Jxw')
#os.chdir('C:\\Users\\Administrator\\Desktop\\Data\\Loan Prediction Analytics Vidhya')
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

In [228]:
dataset=pd.read_csv('train.csv')
dataset1=dataset

In [52]:
dataset.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,03-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,26-09-18,6,1998,1,1,0,0,0,0,598,I-Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,01-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,26-10-18,6,1998,1,1,0,0,0,0,305,L-Very High Risk,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,26-09-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1


In [53]:
import datetime
x = datetime.datetime.now()
currentyear=int(x.strftime("%Y"))
#currentyear=int(currentyear)
currentyear
dataset['DateofBirth']=pd.to_datetime(dataset['Date.of.Birth'])
dataset['year'] = dataset['DateofBirth'].dt.year
dataset['age']=[currentyear-(int(dataset['year'][i])-100) if dataset['year'][i]>2000 else currentyear-int(dataset['year'][i]) for i in range(len(dataset['year']))]

In [54]:
Frequency_state_id={4: 19.244790996508744,
 3: 14.616090652530087,
 6: 14.370330339603868,
 13: 7.6704667301440255,
 9: 6.871852938401228,
 8: 6.089108486236565,
 5: 4.364926186125908,
 14: 4.0376746699606265,
 1: 3.8326599586539367,
 7: 2.910522658843511,
 11: 2.882644089314359,
 18: 2.3212125891041975,
 15: 2.165521500810623,
 12: 1.8056735033497175,
 2: 1.784228449865754,
 17: 1.7117441690899577,
 10: 1.5461883561937604,
 16: 1.151599372088834,
 19: 0.44391260711804215,
 20: 0.07934669789066454,
 21: 0.06690856686996577,
 22: 0.03259648129562435}

In [55]:
dataset['State_ID']=[99 if Frequency_state_id[dataset['State_ID'][i]]<5 else dataset['State_ID'][i] for i in range(len(dataset['State_ID']))]

In [56]:
Group={'C-Very Low Risk':'Very Low Risk','A-Very Low Risk':'Very Low Risk','D-Very Low Risk':'Very Low Risk',
       'B-Very Low Risk':'Very Low Risk','F-Low Risk':'Low Risk','E-Low Risk':'Low Risk',
        'G-Low Risk':'Low Risk','K-High Risk':'High Risk','J-High Risk':'High Risk',
       'Not Scored: Sufficient History Not Available':'Not Scored',
       'Not Scored: Not Enough Info available on the customer':'Not Scored',
       'Not Scored: No Activity seen on the customer (Inactive)':'Not Scored',
       'Not Scored: No Updates available in last 36 months':'Not Scored',
       'Not Scored: Only a Guarantor':'Not Scored',
       'Not Scored: More than 50 active Accounts found':'Not Scored',
       'No Bureau History Available':'No History',
       'H-Medium Risk':'Medium Risk','I-Medium Risk':'Medium Risk','L-Very High Risk':'Very High Risk',
       'M-Very High Risk':'Very High Risk'}
dataset['PERFORM_CNS.SCORE.DESCRIPTION']=dataset['PERFORM_CNS.SCORE.DESCRIPTION'].map(Group)

In [68]:
dataset['PRI.NO.OF.ACCTS']=['<lessthan3' if dataset['PRI.NO.OF.ACCTS'][i]<=3 else '3to7' if dataset['PRI.NO.OF.ACCTS'][i]>3 and dataset['PRI.NO.OF.ACCTS'][i]<=7 else 'greaterthan7' for i in range(len(dataset['PRI.NO.OF.ACCTS']))]

In [69]:
dataset['PRI.ACTIVE.ACCTS']=['1' if dataset['PRI.ACTIVE.ACCTS'][i]<=1 else '2' if dataset['PRI.ACTIVE.ACCTS'][i]>1 and dataset['PRI.ACTIVE.ACCTS'][i]<=2 else 'greaterthan2' for i in range(len(dataset['PRI.ACTIVE.ACCTS']))]

In [70]:
dataset['PRI.OVERDUE.ACCTS']=['0' if dataset['PRI.OVERDUE.ACCTS'][i]<1 else '1to5' if dataset['PRI.OVERDUE.ACCTS'][i]>=1 and dataset['PRI.OVERDUE.ACCTS'][i]<=5 else 'greaterthan5' for i in range(len(dataset['PRI.OVERDUE.ACCTS']))]

In [60]:
dataset['morethansanctioned']=['yes' if dataset['PRI.DISBURSED.AMOUNT'][i]>dataset['PRI.SANCTIONED.AMOUNT'][i] else 'no' for i in range(len(dataset['PRI.SANCTIONED.AMOUNT']))]

In [61]:

dataset['NEW.ACCTS.IN.LAST.SIX.MONTHS']=[0 if dataset['NEW.ACCTS.IN.LAST.SIX.MONTHS'][i]==0 else 1 for i in range(len(dataset['NEW.ACCTS.IN.LAST.SIX.MONTHS']))]

In [62]:
dataset['DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS']=[1 if dataset['DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS'][i]==0 else 1 for i in range(len(dataset['DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS']))]

In [63]:
dataset['AVERAGE.ACCT.AGE']=[dataset['AVERAGE.ACCT.AGE'][i].replace('yrs','') for i in range(len(dataset['AVERAGE.ACCT.AGE']))]
dataset['AVERAGE.ACCT.AGE']=[dataset['AVERAGE.ACCT.AGE'][i].replace('mon','') for i in range(len(dataset['AVERAGE.ACCT.AGE']))]
list_age=[]
for i in range(len(dataset['AVERAGE.ACCT.AGE'])):
    b=[]
    for s in dataset['AVERAGE.ACCT.AGE'][i].split():
        if s.isdigit():
            b.append(s)
    age=np.add(np.multiply(int(b[0]),12),int(b[1]))
    list_age.append(age)  
dataset['AVERAGE.ACCT.AGE']=list_age

In [64]:
dataset['CREDIT.HISTORY.LENGTH']=[dataset['CREDIT.HISTORY.LENGTH'][i].replace('yrs','') for i in range(len(dataset['CREDIT.HISTORY.LENGTH']))]
dataset['CREDIT.HISTORY.LENGTH']=[dataset['CREDIT.HISTORY.LENGTH'][i].replace('mon','') for i in range(len(dataset['CREDIT.HISTORY.LENGTH']))]
list_age=[]
for i in range(len(dataset['CREDIT.HISTORY.LENGTH'])):
    b=[]
    for s in dataset['CREDIT.HISTORY.LENGTH'][i].split():
        if s.isdigit():
            b.append(s)
    age=np.add(np.multiply(int(b[0]),12),int(b[1]))
    list_age.append(age)  
dataset['CREDIT.HISTORY.LENGTH']=list_age

In [204]:
dataset.head()

,ltv,State_ID,Aadhar_flag,VoterID_flag,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,loan_default
0,89.55,6,1,0,No Bureau History Available,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
1,73.23,6,1,0,I-Medium Risk,1,1,1,0,1,1yrs 11mon,1yrs 11mon,1
2,89.63,6,1,0,No Bureau History Available,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
3,88.48,6,1,0,L-Very High Risk,3,0,0,0,0,0yrs 8mon,1yrs 3mon,1
4,88.39,6,1,0,No Bureau History Available,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1


In [216]:
nominal=['Aadhar_flag',
 'VoterID_flag',
 'PERFORM_CNS.SCORE.DESCRIPTION',
 'PRI.NO.OF.ACCTS',
 'PRI.ACTIVE.ACCTS',
 'PRI.OVERDUE.ACCTS',
 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS']

In [224]:
from sklearn.preprocessing import LabelBinarizer
from collections import defaultdict
import pandas as pd
encoding_binarizer=defaultdict(LabelBinarizer)
for feat in nominal:
    encoded_df=pd.DataFrame()
    d=pd.DataFrame(encoding_binarizer[feat].fit_transform(dataset[feat]))
    test_column=d.columns.values
    list_columns=[feat+'_'+str(j) for j in test_column]
    #print(list_columns)
    d.columns=list_columns
    #print(d.head())
    encoded_df=pd.concat([encoded_df,d],axis=1).reset_index(drop=True)
    dataset.drop(feat,axis=1,inplace=True)
    dataset=dataset.reset_index(drop=True)
    dataset=pd.concat([dataset,encoded_df],axis=1)
#with open('C:\\Users\\prudi\\Desktop\\Data Sets\\Loan Prediction\\binarizer_pickle.pkl','wb') as f:
#    pickle.dump(encoding_binarizer,f)
with open('C:\\Users\\prudi\\Documents\\GitHub\\Code\\2019\\Use Cases\\Loan Prediction\\lablebinariser_pickle.pkl','wb') as f:
    pickle.dump(encoding_binarizer,f)
Labelencoderpickle_in = open("C:\\Users\\prudi\\Documents\\GitHub\\Code\\2019\\Use Cases\\Loan Prediction\\lablebinariser_pickle.pkl","rb")
Labelencoder_dict = pickle.load(Labelencoderpickle_in)

In [231]:
from sklearn.preprocessing import LabelBinarizer
from collections import defaultdict
import pandas as pd
for feat in nominal:
    encoded_df=pd.DataFrame()
    d=pd.DataFrame(encoding_binarizer[feat].transform(dataset[feat]))
    test_column=d.columns.values
    list_columns=[feat+'_'+str(j) for j in test_column]
    #print(list_columns)
    d.columns=list_columns
    #print(d.head())
    encoded_df=pd.concat([encoded_df,d],axis=1).reset_index(drop=True)
    dataset.drop(feat,axis=1,inplace=True)
    dataset=dataset.reset_index(drop=True)
    dataset=pd.concat([dataset,encoded_df],axis=1)
#with open('C:\\Users\\prudi\\Desktop\\Data Sets\\Loan Prediction\\binarizer_pickle.pkl','wb') as f:
#    pickle.dump(encoding_binarizer,f)
Labelencoderpickle_in = open("C:\\Users\\prudi\\Documents\\GitHub\\Code\\2019\\Use Cases\\Loan Prediction\\lablebinariser_pickle.pkl","rb")
Labelencoder_dict = pickle.load(Labelencoderpickle_in)

In [232]:
dataset['P']

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,PAN_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default,Aadhar_flag_0,VoterID_flag_0,PERFORM_CNS.SCORE.DESCRIPTION_0,PERFORM_CNS.SCORE.DESCRIPTION_1,PERFORM_CNS.SCORE.DESCRIPTION_2,PERFORM_CNS.SCORE.DESCRIPTION_3,PERFORM_CNS.SCORE.DESCRIPTION_4,PERFORM_CNS.SCORE.DESCRIPTION_5,PERFORM_CNS.SCORE.DESCRIPTION_6,PERFORM_CNS.SCORE.DESCRIPTION_7,PERFORM_CNS.SCORE.DESCRIPTION_8,PERFORM_CNS.SCORE.DESCRIPTION_9,PERFORM_CNS.SCORE.DESCRIPTION_10,PERFORM_CNS.SCORE.DESCRIPTION_11,PERFORM_CNS.SCORE.DESCRIPTION_12,PERFORM_CNS.SCORE.DESCRIPTION_13,PERFORM_CNS.SCORE.DESCRIPTION_14,PERFORM_CNS.SCORE.DESCRIPTION_15,PERFORM_CNS.SCORE.DESCRIPTION_16,PERFORM_CNS.SCORE.DESCRIPTION_17,PERFORM_CNS.SCORE.DESCRIPTION_18,PERFORM_CNS.SCORE.DESCRIPTION_19,PRI.NO.OF.ACCTS_0,PRI.NO.OF.ACCTS_1,PRI.NO.OF.ACCTS_2,PRI.NO.OF.ACCTS_3,PRI.NO.OF.ACCTS_4,PRI.NO.OF.ACCTS_5,PRI.NO.OF.ACCTS_6,PRI.NO.OF.ACCTS_7,PRI.NO.OF.ACCTS_8,PRI.NO.OF.ACCTS_9,PRI.NO.OF.ACCTS_10,PRI.NO.OF.ACCTS_11,PRI.NO.OF.ACCTS_12,PRI.NO.OF.ACCTS_13,PRI.NO.OF.ACCTS_14,PRI.NO.OF.ACCTS_15,PRI.NO.OF.ACCTS_16,PRI.NO.OF.ACCTS_17,PRI.NO.OF.ACCTS_18,PRI.NO.OF.ACCTS_19,PRI.NO.OF.ACCTS_20,PRI.NO.OF.ACCTS_21,PRI.NO.OF.ACCTS_22,PRI.NO.OF.ACCTS_23,PRI.NO.OF.ACCTS_24,PRI.NO.OF.ACCTS_25,PRI.NO.OF.ACCTS_26,PRI.NO.OF.ACCTS_27,PRI.NO.OF.ACCTS_28,PRI.NO.OF.ACCTS_29,PRI.NO.OF.ACCTS_30,PRI.NO.OF.ACCTS_31,PRI.NO.OF.ACCTS_32,PRI.NO.OF.ACCTS_33,PRI.NO.OF.ACCTS_34,PRI.NO.OF.ACCTS_35,PRI.NO.OF.ACCTS_36,PRI.NO.OF.ACCTS_37,PRI.NO.OF.ACCTS_38,PRI.NO.OF.ACCTS_39,PRI.NO.OF.ACCTS_40,PRI.NO.OF.ACCTS_41,PRI.NO.OF.ACCTS_42,PRI.NO.OF.ACCTS_43,PRI.NO.OF.ACCTS_44,PRI.NO.OF.ACCTS_45,PRI.NO.OF.ACCTS_46,PRI.NO.OF.ACCTS_47,PRI.NO.OF.ACCTS_48,PRI.NO.OF.ACCTS_49,PRI.NO.OF.ACCTS_50,PRI.NO.OF.ACCTS_51,PRI.NO.OF.ACCTS_52,PRI.NO.OF.ACCTS_53,PRI.NO.OF.ACCTS_54,PRI.NO.OF.ACCTS_55,PRI.NO.OF.ACCTS_56,PRI.NO.OF.ACCTS_57,PRI.NO.OF.ACCTS_58,PRI.NO.OF.ACCTS_59,PRI.NO.OF.ACCTS_60,PRI.NO.OF.ACCTS_61,PRI.NO.OF.ACCTS_62,PRI.NO.OF.ACCTS_63,PRI.NO.OF.ACCTS_64,PRI.NO.OF.ACCTS_65,PRI.NO.OF.ACCTS_66,PRI.NO.OF.ACCTS_67,PRI.NO.OF.ACCTS_68,PRI.NO.OF.ACCTS_69,PRI.NO.OF.ACCTS_70,PRI.NO.OF.ACCTS_71,PRI.NO.OF.ACCTS_72,PRI.NO.OF.ACCTS_73,PRI.NO.OF.ACCTS_74,PRI.NO.OF.ACCTS_75,PRI.NO.OF.ACCTS_76,PRI.NO.OF.ACCTS_77,PRI.NO.OF.ACCTS_78,PRI.NO.OF.ACCTS_79,PRI.NO.OF.ACCTS_80,PRI.NO.OF.ACCTS_81,PRI.NO.OF.ACCTS_82,PRI.NO.OF.ACCTS_83,PRI.NO.OF.ACCTS_84,PRI.NO.OF.ACCTS_85,PRI.NO.OF.ACCTS_86,PRI.NO.OF.ACCTS_87,PRI.NO.OF.ACCTS_88,PRI.NO.OF.ACCTS_89,PRI.NO.OF.ACCTS_90,PRI.NO.OF.ACCTS_91,PRI.NO.OF.ACCTS_92,PRI.NO.OF.ACCTS_93,PRI.NO.OF.ACCTS_94,PRI.NO.OF.ACCTS_95,PRI.NO.OF.ACCTS_96,PRI.NO.OF.ACCTS_97,PRI.NO.OF.ACCTS_98,PRI.NO.OF.ACCTS_99,PRI.NO.OF.ACCTS_100,PRI.NO.OF.ACCTS_101,PRI.NO.OF.ACCTS_102,PRI.NO.OF.ACCTS_103,PRI.NO.OF.ACCTS_104,PRI.NO.OF.ACCTS_105,PRI.NO.OF.ACCTS_106,PRI.NO.OF.ACCTS_107,PRI.ACTIVE.ACCTS_0,PRI.ACTIVE.ACCTS_1,PRI.ACTIVE.ACCTS_2,PRI.ACTIVE.ACCTS_3,PRI.ACTIVE.ACCTS_4,PRI.ACTIVE.ACCTS_5,PRI.ACTIVE.ACCTS_6,PRI.ACTIVE.ACCTS_7,PRI.ACTIVE.ACCTS_8,PRI.ACTIVE.ACCTS_9,PRI.ACTIVE.ACCTS_10,PRI.ACTIVE.ACCTS_11,PRI.ACTIVE.ACCTS_12,PRI.ACTIVE.ACCTS_13,PRI.ACTIVE.ACCTS_14,PRI.ACTIVE.ACCTS_15,PRI.ACTIVE.ACCTS_16,PRI.ACTIVE.ACCTS_17,PRI.ACTIVE.ACCTS_18,PRI.ACTIVE.ACCTS_19,PRI.ACTIVE.ACCTS_20,PRI.ACTIVE.ACCTS_21,PRI.ACTIVE.ACCTS_22,PRI.ACTIVE.ACCTS_23,PRI.ACTIVE.ACCTS_24,PRI.ACTIVE.ACCTS_25,PRI.ACTIVE.ACCTS_26,PRI.ACTIVE.ACCTS_27,PRI.ACTIVE.ACCTS_28,PRI.ACTIVE.ACCTS_29,PRI.ACTIVE.ACCTS_30,PRI.ACTIVE.ACCTS_31,PRI.ACTIVE.ACCTS_32,PRI.ACTIVE.ACCTS_33,PRI.ACTIVE.ACCTS_34,PRI.ACTIVE.ACCTS_35,PRI.ACTIVE.ACCTS_36,PRI.ACTIVE.ACCTS_37,P

In [193]:
dataset=dataset.drop(columns=['UniqueID','disbursed_amount','asset_cost','branch_id','supplier_id','manufacturer_id','Current_pincode_ID','Date.of.Birth','Employment.Type','DisbursalDate','Employee_code_ID','MobileNo_Avl_Flag','PAN_flag','Driving_flag','Passport_flag','PERFORM_CNS.SCORE','PRI.CURRENT.BALANCE','PRI.SANCTIONED.AMOUNT','PRI.DISBURSED.AMOUNT','SEC.NO.OF.ACCTS','SEC.ACTIVE.ACCTS','SEC.OVERDUE.ACCTS','SEC.CURRENT.BALANCE','SEC.SANCTIONED.AMOUNT','SEC.DISBURSED.AMOUNT','PRIMARY.INSTAL.AMT','SEC.INSTAL.AMT','NO.OF_INQUIRIES'])